# Water Quality Parameters - istSOS upload

The current notebook introduces the computation and ingestion of the aggregate descriptive statistics, on a basins level, of the Water Quality Paramenters produced under the framework of SIMILE project into the [istSOS](http://www.istsos.org/) platform. Additional information on the istSOS capabilities and REST API can be found in its [documentation](http://istsos.org/en/latest/doc/) page.

## 0. Setup Working Environment

### Working libraries and Notebook setup

In [1]:
# Styling notebook

# System
import os
import sys

# Import scripts libraries for the project
sys.path.append('./src/python')


# Import the function to update the notebook style
from nbConfig import (css_styling)

css_styling()

In [2]:
# Data
import numpy as np
import pandas as pd
import pytz
import json
import requests

# Spatial Data
import geopandas as gpd
import rasterio
from rasterstats import zonal_stats

# Data visualization
import matplotlib.pyplot as plt
import seaborn as sns
from rasterio.plot import show_hist

# Import custom libraries
import wqp_istSOS as istsos
import wqpFunctions as wqp

# Widgets
import ipywidgets as wg

In [3]:
# Current Working Directories
cwd = {
    'local': '.',
    'in': './out/OutlierRejection/',
    'out': './out/istSOS/',
    'vector': './vector',
}

### istSOS client authentication

In SIMILE, two different instances of istSOS have been implemented for the data upload depending on the lakes. One for lake Lugano, and a second for Lake Como and Maggiore. For this reason, it is required specifying the proper hosting endpoint, service and credentials to trigger the different procedures fot data ingestion.

The hosting endpoints for each instance are the following:
- Lake Lugano: `'https://istsos.ddns.net/'`
- Lake Como and Maggiore: `'http://150.145.35.198:2000'`

The user credentials can be found in the following env files for each lake:
- Lake Lugano: `'../env_lugano.json'`
- Lake Como and Maggiore: `'../env_como_maggiore.json'`

The service for each lake is the following:
- Lake Lugano: `'ceresiohourly'`
- Lake Como: `'lariolive'`
- Lake Maggiore: `'maggiorelive'`

**Note:** The list of names for the procedures are provided in `./src/python/wqp_istSOS.py`

In [6]:
c = istsos.istSOSClient(HOST='https://istsos.ddns.net',
                        HEADERS=istsos.istSOSClient.HEADERS,
                        SERVICE='ceresiohourly',
                        PROCEDURE_LAKE='LUGANO',
                        ENV_FILE='../env_lugano.json')

The defined user contains the following information to reach istSOS for the data upload

In [7]:
c.__dict__

{'payload': {'grant_type': 'password',
  'client_id': 'istsos-istsos',
  'username': 'satellite',
  'password': 'H!gchYY4'},
 'headers': {'Content-Type': 'application/x-www-form-urlencoded'},
 'tokenUrl': 'https://istsos.ddns.net/auth/realms/istsos/protocol/openid-connect/token',
 'apiEndpoint': 'https://istsos.ddns.net/istsos/wa/istsos/services/ceresiohourly',
 'procedures': ['SATELLITE_CHL_TURB_LUG_NORTH',
  'SATELLITE_CHL_TURB_LUG_SOUTH',
  'SATELLITE_TEMP_LUG_NORTH',
  'SATELLITE_TEMP_LUG_SOUTH']}

In order to trigger the different procedures for data ingestion into the plaftorm it is required to retrive its corresponding **id**.

In [8]:
# Retrieve the id for the procedure using the curent client session in istSOS with the provided credentials.
c.updateBearerToken() # Make sure that the current token for the user is still valid.
assigned_sensor_id = c.getProcedureIDs()

<Response [200]>
SATELLITE_CHL_TURB_LUG_NORTH
SATELLITE_CHL_TURB_LUG_SOUTH
SATELLITE_TEMP_LUG_NORTH
SATELLITE_TEMP_LUG_SOUTH


In [ ]:
a = requests.get('http://150.145.35.198:2000/',timeout = 100)

In [ ]:
assigned_sensor_id

## 1. Vector Data - Basins

In this section, we import the vector layer for the computation of the aggregate statistics, based in the features of the imported layer. In this case, for the division of the lakes ata a basing level.

In [ ]:
!pip install matplotlib_scalebar
from matplotlib_scalebar.scalebar import ScaleBar

In [ ]:
# Layer dataset
gdf_basins = gpd.read_file(os.path.join(cwd['vector'],'simile_basins','simile_basins.shp'))

In [ ]:
gdf_basins

In [ ]:
fig ,ax = plt.subplots(1,1,figsize=(10,10))
gdf_basins.plot(ax = ax, edgecolor="black", alpha = 0.3, column = 'LakeProc', legend = True,\
               legend_kwds={'loc': 'center left', 'bbox_to_anchor':(1,0.5)})
ax.grid()

scale1 = ScaleBar(
    dx=1,
    location='lower right',  # in relation to the whole plot
    label_loc='left', scale_loc='bottom'  # in relation to the line
)
ax.add_artist(scale1)

## 2. Compute Descriptive Statistics - Basin Level

In the following section, it will be possible to select the procedure used for the data upload. After selecting the procedure of interest, a sample of the data format will be requested, this sample will serve for the update of new statistics upload.

### Procedure Selection - Data Formatting for upload

In [10]:
procedure = wg.Select(
    options = c.procedures,
    value = c.procedures[0],
    description = 'Select the procedure:',
    disabled = False,
    style = {
        'description_width': 'auto'
    },
    layout = wg.Layout(width='50%', height='100px'),
)

In [11]:
sensor = wg.Select(
    options = ['S3', 'L8'],
    value = 'S3',
    description = 'Select the sensor:',
    disabled = False,
    style = {
        'description_width': 'auto'
    },
    layout = wg.Layout(width='50%', height='100px'),
)

Select the procedure of interest to extract/ingest data into the selected istSOS instance.

The sensor selection accounts for a further step in obtaining estimates for the Water Quality Parameters of Interest.

In [12]:
wg.HBox([procedure, sensor])

In [13]:
c.updateBearerToken()
data = c.getRequestSample(procedure.value)

<Response [200]>
GetObservation requested successfully executed


In [14]:
# Sample observation for the selected procedure
data

{'name': 'SATELLITE_CHL_TURB_LUG_NORTH',
 'samplingTime': {'beginPosition': '2019-12-10T09:39:19+00:00',
  'endPosition': '2022-05-26T09:58:18+00:00',
  'duration': 'P898DT18M59S'},
 'procedure': 'urn:ogc:def:procedure:x-istsos:1.0:SATELLITE_CHL_TURB_LUG_NORTH',
 'observedProperty': {'CompositePhenomenon': {'id': 'comp_88',
   'dimension': '5',
   'name': 'timeSeriesOfObservations'},
  'component': ['urn:ogc:def:parameter:x-istsos:1.0:time:iso8601',
   'urn:ogc:def:parameter:x-istsos:1.0:water:Chl:a',
   'urn:ogc:def:parameter:x-istsos:1.0:water:Chl:a:qualityIndex',
   'urn:ogc:def:parameter:x-istsos:1.0:water:TSS',
   'urn:ogc:def:parameter:x-istsos:1.0:water:TSS:qualityIndex']},
 'featureOfInterest': {'name': 'urn:ogc:def:feature:x-istsos:1.0:Point:lugano_basin_north',
  'geom': "<gml:Point srsName='EPSG:4326'><gml:coordinates>9.01129783380814,46.0036237146487,270.5</gml:coordinates></gml:Point>"},
 'result': {'DataArray': {'elementCount': '5',
   'field': [{'name': 'Time',
     'def

### Export Basin Stats to CSV files

For each of the different WQP, multiple products have been extracted. In general, the stats are computed for the WQP maps that passed through the outlier detection/rejection process with the 3$\sigma$ filter.

In [ ]:
# Typology of the products
typology_list = [
    'chl',
    'tsm',
    'lswt'
]
# Outlie Rejection methods
or_list = [
    'IQR',
    '2Sigma',
    '3Sigma'
]

# Subset to the buffer size (same used as for the coregistration)
featureGeometry = os.path.join(cwd['vector'],'simile_laghi_extent/simile_laghi_extent_600m_buff.shp')

In [ ]:
typology = wg.Select(
    options = typology_list,
    value = typology_list[0],
    description = 'Select the WQP type:',
    disabled = False,
    style = {
        'description_width': 'auto'
    },
    layout = wg.Layout(width='50%', height='100px'),
)

orMethod = wg.Select(
    options = or_list,
    value = or_list[0],
    description = 'Select the OR method:',
    disabled = False,
    style = {
        'description_width': 'auto'
    },
    layout = wg.Layout(width='50%', height='100px'),
)

# Widgets
wg.HBox([typology, orMethod])

In [ ]:
in_path = os.path.join(cwd['in'],'wqp',orMethod.value,typology.value)
in_path

In [ ]:
in_path = os.path.join(cwd['in'],'wqp',orMethod.value,typology.value)

# In case it is required cropping the maps

# out_path = os.path.join(in_path,'cropped')
# shared_maps = list(set(os.listdir(in_path)).intersection(os.listdir(out_path)))
# missing_maps = list(set(os.listdir(in_path)) ^ set(os.listdir(out_path)))
# print(missing_maps)
# for root, dirs, files in os.walk(in_path):
#     for f in missing_maps:
#         if ((f.endswith('.tif')) & (dirs==['cropped'])):
#             chl_map = os.path.join(in_path,f)
#             print(chl_map)
#             wqp_map = wqp.wqp(chl_map)
#             wqp_map.readWQP()
#             wqp_map.cropRasterByFeatures(featureGeometry,'Nome')
#             d_temp = wqp_map.crops['wqp']['crop']
#             d_temp[d_temp<0] = 0
#             wqp_map.crops['wqp']['crop'] = d_temp
#             wqp_map.saveMaskedImage(os.path.join(out_path,wqp_map.name+'.tif'),'wqp',band=1)
#             wqp_map.image.close()

In [ ]:
# Take into account that some of the files have already been analyzed
stats = os.path.join(in_path,f'lakesStats_{typology.value}.csv')
print(in_path)
print(stats)
data = []
for root, dirs, files in os.walk(in_path):
    if os.path.exists(stats):
        df = pd.read_csv(stats)
        stats_file_names = set(list(df['name']+'.tif'))
        missing_stats = list(stats_file_names ^ set(os.listdir(in_path)))
    else:
        missing_stats = files
    for file in missing_stats:
        if ((file.endswith('.tif')) & (root.split('\\')[-1]!='outliers')):
            print(root,file)
            f = os.path.join(root,file)
            
            # Read file
            src = wqp.wqp(f)
            src.readWQP()
            src.name

            # Compute statistics for the basins polygons
            src.computeStatistics(gdf_basins, 'LakeProc',"count min mean max median std  percentile_25 percentile_50 percentile_75", 0)

            # Format output
            data.append(wqp.wqp.exportWQPFormatStats(src))

df = pd.concat(data)
# Add values to csv file
istsos.appendStatsFile(df, stats)

### istSOS JSON format

In case of S3, it is needed to import both reference data files for the computed statistics

In [ ]:
# Load the statistics files
file_name = os.path.join(in_path,f'lakesStats_{typology.value}_submit.csv')
df = pd.read_csv(file_name,delimiter=";")
df.date = pd.to_datetime(df.date)

In [ ]:
df = df.dropna()
df = df.sort_values(by=['date'])
df = df.round(2)
df.shape

In [ ]:
df.dropna(inplace=True)
df.shape

In [ ]:
df.head()

In [ ]:
max(df.date)

In [ ]:
procedure.value

In [ ]:
#TODO: update the updateDataRequest and resultsWQPvalues functions for the new procedures naming convention
a = istsos.updateDataRequest(df=df,
                    dataSample=data, 
                    WQP_DEFINITIONS=istsos.istSOSClient.WQP_DEFINITIONS,
                    WQP_PROCEDURES=istsos.istSOSClient.WQP_PROCEDURES,
                    PROCEDURE=procedure.value)

In [ ]:
a

In [ ]:
# # Get Observation
# SERVICE_NAME = 'demo'
# OFFERING_NAME = 'temporary'
# PROCEDURE_NAME = 'SATELLITE_CHL_TURB_CO_EAST'
# DEFINITION_URNs = 'urn:ogc:def:parameter:x-istsos:1.0:water:Chl'
# BEGIN_TIME = "2020-01-01T00:01:00+01"
# END_TIME = "2020-01-17T00:01:00+01"

# url_base = "https://istsos.ddns.net/istsos/"
# url_test =f"{url_base}/wa/istsos/services/{SERVICE_NAME}/operations/getobservation/offerings/{OFFERING_NAME}/procedures/{PROCEDURE_NAME}/observedproperties/{DEFINITION_URNs}/eventtime/{BEGIN_TIME}/{END_TIME}"
# resp = requests.get(url_test, headers=c.requestHeaders)
# resp.text


In [ ]:
# WQP_PROCEDURES = ['CHL','TURB','WT']
# a = procedure.value.split('_')
# wqps = list(set(a).intersection(WQP_PROCEDURES))


In [ ]:

# # INSERT OBSERVATION PARAMETERS

# ########## PARAMETERS TO BE EDITED ####################
# LAKE = 'CO' # {CO: COMO, LU: LUGANO, MA: MAGGIORE}
# WQP_INPUT = ['LSWT'] #['LSWT','CHL','TSM']
# #######################################################

# DATE_START = pytz.utc.localize(df.date.min()).isoformat().replace('+00:00','Z')
# DATE_END = pytz.utc.localize(df.date.max()).isoformat().replace('+00:00','Z')


# if LAKE == 'CO':
#     BASINS_LIST = ['CO_NORTH','CO_EAST','CO_WEST']
# elif LAKE == 'LU':
#     BASINS_LIST = ['LU_EAST','LU_WEST']
# elif LAKE == 'MA':
#     BASINS_LIST = ['MA_ALL']
    
# # Observed properties
# OP_COMPOSITE_PHENOMENON_ID = 'comp_1' #random name?
# OP_COMPOSITE_PHENOMENON_DIMENSION = '{}'.format(len(WQP_INPUT)+1) #int number of input observed variables (not observations)
# OP_COMPONENT = ["urn:ogc:def:parameter:x-istsos:1.0:time:iso8601"]
# for wqp in WQP_INPUT: #Observed parameters names: {'lswt':'water-temperature';'chl'}
#     if 'LSWT' in wqp:
#         OP_COMPONENT.append("urn:ogc:def:parameter:x-istsos:1.0:water-temperature")
#     elif 'CHL' in wqp:
#         OP_COMPONENT.append("urn:ogc:def:parameter:x-istsos:1.0:water:Chl")
#     elif 'TSM' in wqp:
#         OP_COMPONENT.append("urn:ogc:def:parameter:x-istsos:1.0:water:Turb")

# #Results
# RES_DA_ELEMENT_COUNT = OP_COMPOSITE_PHENOMENON_DIMENSION # Does this parameter have the same magnitude as the composite dimension?
# RES_DA_FIELDS = [
#     {
#         "name": "Time",
#         "definition": "urn:ogc:def:parameter:x-istsos:1.0:time:iso8601"
#     },
# ]
# for wqp in WQP_INPUT: #Observed parameters names: {'lswt':'water-temperature';'chl'}
#     if 'LSWT' == wqp:
#         RES_DA_FIELDS.append(
#             {
#                 "name": "water-temperature",
#                 "definition":"urn:ogc:def:parameter:x-istsos:1.0:water-temperature",
#                 "uom":"\u00b0C"
#             }
#         )
#     elif 'CHL' == wqp:
#         RES_DA_FIELDS.append(
#             {
#                 "name": "water-Chl",
#                 "definition":"urn:ogc:def:parameter:x-istsos:1.0:water:Chl",
#                 "uom":"mg/m^3"
#             }
#         )
#     elif 'TSM' == wqp:
#         RES_DA_FIELDS.append(
#             {
#                 "name": "water-Turb",
#                 "definition":"urn:ogc:def:parameter:x-istsos:1.0:water:Turb",
#                 "uom":"g/m^3"
#             }
#         )

# # Create insertObsevartion model for istSOS depending on the procedure.
# for PROCEDURE in PROCEDURES_LIST:
#     obs = {
#         "AssignedSensorId":f"{ASSIGNED_SENSOR_ID[PROCEDURE]}",
#         "ForceInsert":"false",
#         "Observation":{
#             "name":f"{PROCEDURE}",
#             "samplingTime":{
#             "beginPosition":f"{DATE_START}",
#             "endPosition":f"{DATE_END}"
#             },
#             "procedure":f"urn:ogc:def:procedure:x-istsos:1.0:{PROCEDURE}",
#             "observedProperty": {
#                 "CompositePhenomenon":{
#                     "id":f"{OP_COMPOSITE_PHENOMENON_ID}", 
#                     "dimension":f"{OP_COMPOSITE_PHENOMENON_DIMENSION}", 
#                     "name":"timeSeriesOfObservations"
#                 },
#                 "component": OP_COMPONENT
#             },
#             "featureOfInterest":{ # Is it necessary to assign the value to a geometry? 
#                 "name":f"urn:ogc:def:feature:x-istsos:1.0:Point:{PROCEDURE}",
#                 "geom":f"{istsos.getGMLfeature(os.path.join(cwd['out'],'procedures_istSOS'), PROCEDURE+"_point")}" 
#             },
#             "result": {
#                 "DataArray": {
#                     "elementCount":f"{RES_DA_ELEMENT_COUNT}",
#                     "field": RES_DA_FIELDS,
#                     "values": istsos.resultsWQPvalues(df,'_'.join(PROCEDURE.split('_')[-2:])),#BASIN
#                 }
#             }
#         }
#     }
#     print('_'.join(PROCEDURE.split('_')[-2:]))
#     with open(os.path.join(cwd['out'],f"{PROCEDURE}.json"), "w", encoding="UTF-8") as outfile:
#         json.dump(obs, outfile, ensure_ascii=False, indent=4)

### Data Ingestion to istSOS

In [ ]:
obs = {
    "AssignedSensorId":f"{assigned_sensor_id[procedure.value]}",
    "ForceInsert":"true",
    "Observation": a
}

In [ ]:
obs

In [ ]:
print(os.path.join(cwd['out'],f"{procedure.value}.json"))

In [ ]:
with open(os.path.join(cwd['out'],f"{procedure.value}.json"), "w", encoding="UTF-8") as outfile:
    json.dump(obs, outfile, ensure_ascii=False, indent=4)

In [ ]:
c.updateBearerToken()
c.requestHeaders

In [ ]:
json.dumps(obs)

In [ ]:
file_name_json = f'{procedure.value}.json'
filepath = os.path.join(cwd['out'], sensor.value,file_name_json)
print(filepath)
f = open(filepath)

data = json.load(f)
d=json.dumps(data)
d

In [ ]:
c.updateBearerToken()

In [ ]:
#Insert Observation into istSOS test
url = 'https://istsos.ddns.net/istsos/wa/istsos/services/ceresiohourly/operations/insertobservation'
response = requests.post(url,data=json.dumps(obs),headers=c.requestHeaders)
# response = requests.post(url,data=d,headers=c.requestHeaders)
print(response.status_code)
print(response.text)

In [ ]:
data

In [ ]:
url_base = "https://istsos.ddns.net/istsos/wa/istsos/services/ceresiohourly"
# # Sample request. Retrieve al the available procedures in the platform
# url_test = f"{url_base}/procedures/operations/getlist"
url_test =f"{url_base}/procedures/{procedure.value}"
# url_test =f"{url_base}/procedures/SATELLITE_CHL_TURB_CO_NORTH"
resp = requests.get(url_test, data=payload, headers=headers_req)
resp.text

In [ ]:
data

In [ ]:
file_name_json = 'SATELLITE_CHL_TURB_CO_NORTH_2.json'
filepath = os.path.join(cwd['out'], sensor.value,file_name_json)
f = open(filepath)
data = json.load(f)
d=json.dumps(data)
d

### Testing istSOS API  calls

In [ ]:
url_base = "https://istsos.ddns.net/istsos/"
url_test =f"{url_base}/wa/istsos/services/{istsos.istSOSClient.SERVICE}/procedures/operations/getlist"
resp = requests.get(url_test,headers=c.requestHeaders)
resp.text

In [ ]:
url_base = "https://istsos.ddns.net/istsos/"
url_test =f"{url_base}/wa/istsos/services/demo/procedures/operations/geojson"
resp = requests.get(url_test, data=payload, headers=headers_req)
resp.text

In [ ]:
# Get Offerings
url_base = "https://istsos.ddns.net/istsos/"
url_test =f"{url_base}/wa/istsos/services/demo/offerings/operations/getlist"
resp = requests.get(url_test, data=payload, headers=headers_req)
resp.text

In [9]:
# Get Observation
SERVICE_NAME = 'ceresiohourly'
OFFERING_NAME = 'temporary'
PROCEDURE_NAME = procedure.value
# DEFINITION_URNs = 'urn:ogc:def:parameter:x-istsos:1.0:water:Chl:a&urn:ogc:def:parameter:x-istsos:1.0:water:TSS'
DEFINITION_URNs = 'urn:ogc:def:parameter:x-istsos:1.0:water:temperature'
BEGIN_TIME = "2019-01-04T09:54:16Z"
END_TIME = "2022-08-26T09:58:18Z"

url_base = "https://istsos.ddns.net/istsos/"
url_test =f"{url_base}/wa/istsos/services/{SERVICE_NAME}/operations/getobservation/offerings/{OFFERING_NAME}/procedures/{PROCEDURE_NAME}/observedproperties/{DEFINITION_URNs}/eventtime/{BEGIN_TIME}/{END_TIME}"
resp = requests.get(url_test, headers=c.requestHeaders)
resp.text


NameError: name 'procedure' is not defined

In [ ]:
# Get Observation
SERVICE_NAME = 'demo'
OFFERING_NAME = 'temporary'
PROCEDURE_NAME = 'SATELLITE_CHL_TURB_CO_WEST'
DEFINITION_URNs = 'urn:ogc:def:parameter:x-istsos:1.0:water:Chl'
BEGIN_TIME = "2019-01-01T00:01:00+01"
END_TIME = "2022-01-17T00:01:00+01"

url_base = "https://istsos.ddns.net/istsos/"
url_test =f"{url_base}/wa/istsos/services/{SERVICE_NAME}/operations/getobservation/offerings/{OFFERING_NAME}/procedures/{PROCEDURE_NAME}/observedproperties/:/eventtime/last"
resp = requests.get(url_test, headers=c.requestHeaders)
resp.json()


In [ ]:
data = resp.json()

In [ ]:
data['data']